In [1]:
# Importamos las librerías necesarias
import requests
import selenium
import sqlite3
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Creación de la base de datos
con = sqlite3.connect('data/coleccion_anagrama.db')
cursorANA = con.cursor()
cursorANA.execute(
    '''
    CREATE TABLE IF NOT EXISTS anagrama
    (
    isbn VARCHAR,
    ean INT,
    código VARCHAR,
    título TEXT,
    autor TEXT,
    precio FLOAT,
    páginas INT,
    fecha_publicación DATE,
    colección TEXT,
    traducción TEXT,
    sinopsis TEXT
    );
    ''')
con.commit()
con.close()

In [3]:
# Esqueleto del scraper
def scrap_anagrama():
    
    def get_links_colecciones():
        html = requests.get(url_colecciones).content
        soup = BeautifulSoup(html)
        colecciones = {}
        for link in soup.findAll('a'):
            coleccion_link = url_anagrama + str(link.get('href'))
            coleccion = coleccion_link.rsplit('/', 1)[-1].replace('-',' ')
            if "/coleccion/" in coleccion_link:
                colecciones[coleccion] = coleccion_link
        del colecciones['ebooks']
        return colecciones


    def get_links_libros_in_page(url_page):
        html = requests.get(url_page).content
        soup = BeautifulSoup(html)
        links_libros = set([link['href'] for link in soup.find_all(
            "a", class_="book", href=True) if link['href'].startswith('/libro/')])
        return links_libros
        

    def get_libro_info(url_libro):
        '''
        Recoge la información básica del libro
        dada una url de la editorial Anagrama
        y lo carga en un diccionario
        '''
        url_libro = url_libro
        html = requests.get(url_libro).content
        soup = BeautifulSoup(html)
        
        datos_libro = dict()
        
        # Básicos
        datos_libro["TÍTULO"] = soup.h1.get_text()
        datos_libro["AUTOR/A"] = soup.title.get_text().split('-')[1].strip()
        datos_libro["RESUMEN"] = soup.find('div', class_="textContent mt10px").get_text()
        
        # Tabla
        for e in soup.find('table'):
            data = e.find_all('td')
            col, row = data[0].get_text(), data[1].get_text()
            datos_libro[col] = row
        
        # Corrección de la información de COLECCIÓN
        datos_libro["COLECCIÓN"] = url_libro.split('/')[4].replace('-',' ').title()

        return datos_libro
        

    def get_row_values(info_libro):
        columns = ['ISBN', 'EAN', 'TÍTULO', 'AUTOR/A', 'COLECCIÓN', 'CÓDIGO',
                   'NÚM. DE PÁGINAS', 'PUBLICACIÓN', 'PVP CON IVA', 'RESUMEN',
                   'TRADUCCIÓN']
        for key, value in info_libro.items():
            row_values = []
            for column in columns:
                try:
                    row_values.append(info_libro[column])
                except:
                    row_values.append(None)
        return row_values
        

    def sql_insert(con, row_values):
        cursorObj = con.cursor()
        cursorObj.execute('''
                INSERT INTO 
                anagrama(isbn, ean, título, autor, colección, código,
                páginas, fecha_publicación, precio, sinopsis, traducción
                ) 
                VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', row_values)
        con.commit()
    
    
    con = sqlite3.connect('data/coleccion_anagrama.db')

    url_anagrama = 'https://www.anagrama-ed.es'
    url_colecciones = 'https://www.anagrama-ed.es/colecciones'
    
    colecciones = get_links_colecciones()
    
    for coleccion, link in colecciones.items():
        page = 1
        while bool(page)==True:
            url_page = link + '?p=' + str(page)
            listado_links_libros = get_links_libros_in_page(url_page)
            if len(listado_links_libros) != 0:
                for url_libro in listado_links_libros:
                    url_libro = url_anagrama + url_libro
                    info_libro = get_libro_info(url_libro)
                    row_values = get_row_values(info_libro)
                    row_values
                    sql_insert(con, row_values)
                page += 1
            else:
                page = False
    return '¡Listo!'

In [4]:
%time scrap_anagrama()

CPU times: user 3min 17s, sys: 15 s, total: 3min 32s
Wall time: 53min 38s


'¡Listo!'

In [5]:
conn = sqlite3.connect('data/coleccion_anagrama.db', isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)
df = pd.read_sql_query("SELECT * FROM anagrama", conn)
df.to_csv('data/dataset.csv', index=False)

In [6]:
display(df)

,isbn,ean,código,título,autor,precio,páginas,fecha_publicación,colección,traducción,sinopsis
0,978-84-339-8117-2,9788433981172,PN 1077,Golpe de kárate,"Nors, Dorthe",17.9 €,144.0,27/04/2022,Panorama De Narrativas,Victoria Alonso,Quince relatos prodigiosos sobre situaciones a...
1,978-84-339-8108-0,9788433981080,PN 1068,Los colores del adiós,"Schlink, Bernhard",19.9 €,224.0,12/01/2022,Panorama De Narrativas,Juan de Sola,Nueve relatos prodigiosos que exploran la comp...
2,978-84-339-8112-7,9788433981127,PN 1072,"Bueno, aquí estamos","Swift, Graham",18.9 €,184.0,16/02/2022,Panorama De Narrativas,Antonio-Prometeo Moya Valle,Una novela de gran sensibilidad que nos envuel...
3,978-84-339-8111-0,9788433981110,PN 1071,Nacido de ninguna mujer,"Bouysse, Franck",19.9 €,304.0,02/02/2022,Panorama De Narrativas,Rosa Alapont Calderaro,Un manuscrito. Una joven enfrentada a un desti...
4,978-84-339-8116-5,9788433981165,PN 1076,Tinta simpática,"Modiano, Patrick",17.9 €,128.0,02/03/2022,Panorama De Narrativas,María Teresa Gallego Urrutia,Un detective busca a una mujer desaparecida. U...
...,...,...,...,...,...,...,...,...,...,...,...
3814,978-84-339-9893-4,9788433998934,NH 641,Poeta chileno,"Zambra, Alejandro",20.9 €,424.0,04/03/2020,Narrativas Hispanicas,None,Alejandro Zambra vuelve en grande a la novela ...
3815,978-84-339-6855-5,9788433968555,NH 354,Los girasoles ciegos,"Méndez, Alberto",13.9 €,160.0,01/01/2004,Narrativas Hispanicas,None,Este libro es el regreso a las historias reale...
3816,978-84-339-9885-9,9788433998859,NH 636,Nuestra parte de noche,"Enriquez, Mariana",22.9 €,672.0,27/11/2019,Desconocido,None,"La herencia, el deseo de pervivir, la paternid..."
3817,978-84-339-9908-5,9788433999085,NH 653,Las maravillas,"Medel, Elena",17.9 €,228.0,07/10/2020,Narrativas Hispanicas,None,La primera novela de Elena Medel: un recorrido...
